In [14]:
import pandas as pd
import numpy as np
from functools import reduce

import seaborn as sns

import sys
import os

sys.path.append(os.path.relpath("../../huygens"))
sys.path.append(os.path.relpath("../../galileo"))

import galileo as gal
import huygens as huy

In [3]:
def concat_cols(df, cols, delim):
    cols_str = [df[x].astype(str) for x in cols]

    return reduce(lambda a, b: a + delim + b, cols_str)

In [10]:
tcga_genex = pd.read_hdf("../../data/processed/TCGA/TCGA_genex_norm.h5",key="tcga_genex")

normal_genex = tcga_genex[tcga_genex.index.map(lambda x: x[-2:] == "11")]

tcga_genex = tcga_genex[tcga_genex.index.map(lambda x: x[-2:] != "11")]

In [8]:
tcga_splicing = pd.read_hdf("../../data/processed/TCGA/merged.h5",key="tcga_splicing")


tcga_splicing.index = tcga_splicing.index.map(lambda x: x[:15])

tcga_splicing = tcga_splicing[~tcga_splicing.index.duplicated(keep="first")]

tcga_splicing = tcga_splicing[tcga_splicing.index.map(lambda x: x[-2:] != "11")]

# RPL22 CN vs splicing

In [20]:
tcga_cn = pd.read_hdf("../../data/processed/TCGA/tcga_cn.hdf", key="tcga_cn")
tcga_cn = tcga_cn[tcga_cn.index.map(lambda x: x[-2:] != "11")]

In [26]:
rpl22_cn_splicing = gal.mat_corrs_naive(
    tcga_cn["RPL22"], tcga_splicing, method="spearman",pbar=True)

rpl22_cn_splicing.to_hdf(
    "../data/intermediate/rpl22_cn_splicing.h5", key="rpl22_cn_splicing", mode="w")

/Users/khu/anaconda3/envs/cenv/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/khu/anaconda3/envs/cenv/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/khu/anaconda3/envs/cenv/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/Users/khu/anaconda3/envs/cenv/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/Users/khu/anaconda3/envs/cenv/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


# Splicing and expression correlations

## RPL22L1 expression vs splicing

In [27]:
rpl22l1_genex_splicing = gal.mat_corrs_naive(tcga_genex["RPL22L1_15209"],
                                           tcga_splicing,
                                           method="spearman",
                                           pbar=True
                                          )

rpl22l1_genex_splicing.to_hdf("../data/intermediate/rpl22l1_genex_splicing.h5",key="rpl22l1_genex_splicing",mode="w")

In [28]:
rpl22l1_genex_splicing_subtyped = gal.mat_corrs_subtyped(tcga_genex["RPL22L1_15209"],
                                                         tcga_splicing,
                                                         tcga_sample_info["abbreviated_disease"],
                                                         pbar=True,
                                                         stack=True,
                                                         min_count=50,
                                                         method="spearman"
                                                         )

rpl22l1_genex_splicing_subtyped.to_hdf("../data/intermediate/rpl22l1_genex_splicing_subtyped.h5",
                                       key="rpl22l1_genex_splicing_subtyped",
                                       mode="w")

## Splicing vs gene expression

In [29]:
mdm4_splicing_genex = gal.mat_corrs_naive(tcga_splicing["MDM4_ENSG00000198625_ENSG00000198625.8_ES_1_204501318:204501374:204506557:204506625:204507336:204507436_204506557:204506625"],
                                          tcga_genex,
                                          pbar=True,
                                          method="spearman"
                                          )

rpl22l1_splicing_genex = gal.mat_corrs_naive(tcga_splicing["RPL22L1_ENSG00000163584_ENSG00000163584.13_A3_3_170586086:170586168:170585801:170585923:170585801:170585990_170585923:170585990"],
                                             tcga_genex,
                                             pbar=True,
                                             method="spearman"
                                             )

mdm4_splicing_genex.to_hdf(
    "../data/intermediate/mdm4_splicing_genex.h5", key="mdm4_splicing_genex", mode="w")

rpl22l1_splicing_genex.to_hdf(
    "../data/intermediate/rpl22l1_splicing_genex.h5", key="rpl22l1_splicing_genex", mode="w")


In [30]:
mdm4_splicing_genex_subtyped = gal.mat_corrs_subtyped(tcga_splicing["MDM4_ENSG00000198625_ENSG00000198625.8_ES_1_204501318:204501374:204506557:204506625:204507336:204507436_204506557:204506625"],
                                               tcga_genex,
                                               tcga_sample_info["abbreviated_disease"],
                                               pbar=True,
                                               stack=True,
                                               min_count=50,
                                               method="spearman"
                                               )

rpl22l1_splicing_genex_subtyped = gal.mat_corrs_subtyped(tcga_splicing["RPL22L1_ENSG00000163584_ENSG00000163584.13_A3_3_170586086:170586168:170585801:170585923:170585801:170585990_170585923:170585990"],
                                                  tcga_genex,
                                                  tcga_sample_info["abbreviated_disease"],
                                                  pbar=True,
                                                  stack=True,
                                                  min_count=50,
                                                  method="spearman"
                                                  )

mdm4_splicing_genex_subtyped.to_hdf(
    "../data/intermediate/mdm4_splicing_genex_subtyped.h5", key="mdm4_splicing_genex_subtyped", mode="w")
rpl22l1_splicing_genex_subtyped.to_hdf(
    "../data/intermediate/rpl22l1_splicing_genex_subtyped.h5", key="rpl22l1_splicing_genex_subtyped", mode="w")


# Co-splicing correlations

In [31]:
mdm4_cosplicing = gal.mat_corrs_naive(tcga_splicing["MDM4_ENSG00000198625_ENSG00000198625.8_ES_1_204501318:204501374:204506557:204506625:204507336:204507436_204506557:204506625"],
                                      tcga_splicing,
                                      pbar=True,
                                      method="spearman"
                                      )

rpl22l1_cosplicing = gal.mat_corrs_naive(tcga_splicing["RPL22L1_ENSG00000163584_ENSG00000163584.13_A3_3_170586086:170586168:170585801:170585923:170585801:170585990_170585923:170585990"],
                                         tcga_splicing,
                                         pbar=True,
                                         method="spearman"
                                         )

mdm4_cosplicing.to_hdf(
    "../data/intermediate/mdm4_cosplicing.h5", key="mdm4_cosplicing", mode="w")

rpl22l1_cosplicing.to_hdf(
    "../data/intermediate/rpl22l1_cosplicing.h5", key="rpl22l1_cosplicing", mode="w")

In [32]:
mdm4_cosplicing_subtyped = gal.mat_corrs_subtyped(tcga_splicing["MDM4_ENSG00000198625_ENSG00000198625.8_ES_1_204501318:204501374:204506557:204506625:204507336:204507436_204506557:204506625"],
                                                  tcga_splicing,
                                                  tcga_sample_info["abbreviated_disease"],
                                                  pbar=True,
                                                  stack=True,
                                                  min_count=50,
                                                  method="spearman"
                                                  )

rpl22l1_cosplicing_subtyped = gal.mat_corrs_subtyped(tcga_splicing["RPL22L1_ENSG00000163584_ENSG00000163584.13_A3_3_170586086:170586168:170585801:170585923:170585801:170585990_170585923:170585990"],
                                                     tcga_splicing,
                                                     tcga_sample_info["abbreviated_disease"],
                                                     pbar=True,
                                                     stack=True,
                                                     min_count=50,
                                                     method="spearman"
                                                     )

mdm4_cosplicing_subtyped.to_hdf(
    "../data/intermediate/mdm4_cosplicing_subtyped.h5", key="mdm4_cosplicing_subtyped", mode="w")
rpl22l1_cosplicing_subtyped.to_hdf(
    "../data/intermediate/rpl22l1_cosplicing_subtyped.h5", key="rpl22l1_cosplicing_subtyped", mode="w")